In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

plt.rcParams.update({
    "figure.figsize": (15,5)
})

In [2]:
pilatus = pd.read_csv("order111064/order_111064_data.txt", sep=";")
luzern = pd.read_csv("order111065/order_111065_data.txt", sep=";")

# Aufbereitung der Daten

In [3]:
def df_aufbereitung(dataframe, columnnames):


    #Änderung des Zeitformats von Zahlen zu Datetime
    dataframe['time'] = pd.to_datetime(dataframe['time'], format='%Y%m%d')
    dataframe['time'] = pd.to_datetime(dataframe['time'], format='%Y.%m.%d')

    #Neubenennung der Spaltennamen
    dataframe.columns = columnnames


    #Entfernung von nichtvorhandenen Messungen
    dataframe = dataframe.replace('-', np.nan)
    dataframe.dropna(inplace=True)


    #Anpassung des Datatypes der Niederschlagsmenge
    dataframe['Niederschlag Tagessumme [mm]'] = dataframe['Niederschlag Tagessumme [mm]'].astype('float')

    return dataframe



columnnames_pilatus = ['Station','Datum','Temperatur Tagesmaximum [°C]','Temperatur Tagesminimum [°C]','Temperatur Tagesmittel [°C]','Niederschlag Tagessumme [mm]']
columnnames_luzern  = ['Station','Datum','Temperatur Tagesmaximum [°C]','Temperatur Tagesminimum [°C]','Temperatur Tagesmittel [°C]','Niederschlag Tagessumme [mm]','Neuschneehöhe Tagessumme [cm]']

pilatus = df_aufbereitung(pilatus, columnnames_pilatus)
luzern = df_aufbereitung(luzern, columnnames_luzern)


luzern['Neuschneehöhe Tagessumme [cm]'] = luzern['Neuschneehöhe Tagessumme [cm]'].astype('float')



## Schneefall

Unter Voraussetzung, dass die Tagesmitteltemperatur $\leq0$ ist, gilt die Faustregel:

$$Niederschlag * 10 = Schneehöhe$$




Zur Anpassung der Ermittlung der Schneehöhe, verändere folgenden Codeteil der nächsten Zelle
```python 
pilatus['Niederschlag Tagessumme [mm]']*1
```


In [4]:
pilatus['Neuschneehöhe Tagessumme [cm]']  = np.where(pilatus['Temperatur Tagesmittel [°C]']<=0, pilatus['Niederschlag Tagessumme [mm]']*1,0)

# Interaktiver Plot

In [5]:
# from ipywidgets import IntSlider, interact
import plotly.express as px

In [6]:
def plot_schneehoehe (start_year, end_year):
    #Die Daten werden aufgrund des Start- und Endjahrs gestutzt
    start_date = pd.to_datetime(f'{start_year}-01-01')
    end_date = pd.to_datetime(f'{end_year}-12-31')

    mask_luzern = (luzern['Datum']>=start_date) & (luzern['Datum']<=end_date)
    luzern_gestutzt = luzern.loc[mask_luzern]

    mask_pilatus = (pilatus['Datum']>=start_date) & (pilatus['Datum']<=end_date)
    pilatus_gestutzt = pilatus.loc[mask_pilatus]


        
    fig = px.bar(title="Neuschneehöhen aufgrund zweier Wetterstationen im Raum Luzern")
    fig.add_bar(x=pilatus_gestutzt['Datum'], y=pilatus_gestutzt['Neuschneehöhe Tagessumme [cm]'], name='Pilatus Wetterstation',marker={'color':'red'} )
    fig.add_bar(x=luzern_gestutzt['Datum'], y=luzern_gestutzt['Neuschneehöhe Tagessumme [cm]'], name='Luzern Wetterstation',marker={'color':'green'} )
    fig.update_layout(barmode='overlay')
    fig.update_xaxes(title='Datum')
    fig.update_yaxes(title='Neuschneehöhe Tagessumme [cm]')
    fig.show()

In [8]:
##Hier kannst du die Grenzen für den Plot definieren
plot_schneehoehe(2015, 2023)